In [1]:
import pandas as pd
import numpy as np

In [2]:
DATA_PATH = 'C://Users//aman2//Documents//BTProject//btp2.0//data//{}.csv'

def get_stock_data(stock, start_timestamp=None, end_timestamp=None):
    try:
        stock = stock.replace('.NS', '').replace('.BO', '')
        df = pd.read_csv(DATA_PATH.format(stock))

        df['Date'] = pd.to_datetime(df['Date'])
        df['Date'] = (df['Date'].astype('int64')//1e9) + 19800

        df['Date'] = pd.to_datetime(df['Date'], unit='s')
        if start_timestamp and end_timestamp:
            df = df[(df['Date'] >= start_timestamp) & (df['Date'] <= end_timestamp)]
        df = df.set_index('Date')[['Open', 'High', 'Low', 'Close', 'Volume']]
        return df
    except Exception as e:
        raise ValueError(f"Error in reading data for {stock}. Error: {e}")

def get_data(stocks, start_timestamp=None, end_timestamp=None):
    data = {}
    errors = []
    for stock in stocks:
        try:
            data[stock] = get_stock_data(stock, start_timestamp, end_timestamp)
        except Exception as e:
            errors.append(f"Error in reading data for {stock}. Error: {e}")
    return data, errors

In [3]:
portfolio = pd.read_csv('data.csv', sep=',')
portfolio.columns = ['stock', 'weights']

In [4]:
portfolio.head()

,stock,weights
0,RENUKA,-0.198246
1,ALOKINDS,-0.151068
2,SADBHAV,-0.612605
3,TATACHEM,1.000000
4,NIITLTD,0.005578


In [5]:
print('Number of stocks in portfolio: {}'.format(len(portfolio)))

Number of stocks in portfolio: 40


In [6]:
start_timestamp = '01/01/2020'
end_timestamp = '30/6/2023'

start_timestamp = '01/01/2016'
end_timestamp = '01/01/2020'

start_timestamp = pd.to_datetime(start_timestamp, dayfirst=True)
end_timestamp = pd.to_datetime(end_timestamp, dayfirst=True)

df, errors = get_data(portfolio['stock'].values.tolist(), start_timestamp, end_timestamp)
print('Number of stocks in portfolio: {}'.format(len(df)))
print('Errors in reading data for: {}'.format(errors))

Number of stocks in portfolio: 40
Errors in reading data for: []


In [7]:
prices = []
for stock, data in df.items():
    prices.append(data['Close'])
prices_df = pd.concat(prices, axis=1, keys=df.keys())

In [8]:
prices_df.head()

,RENUKA,ALOKINDS,SADBHAV,TATACHEM,NIITLTD,NAVA,SURYAROSNI,JINDALPOLY,HMT,PFC,...,GAIL,COSMOFIRST,DAAWAT,HINDOILEXP,PCBL,CHAMBLFERT,CESC,NLCINDIA,TNPL,SAIL
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,13.70,6.30,339.500702,334.876190,82.529274,74.785812,61.921040,449.910767,51.799999,44.279350,...,52.109303,146.590576,24.570826,36.000000,10.309450,52.719341,35.064480,51.268738,187.372467,41.419281
2016-01-04,14.30,6.25,333.594635,333.361145,80.071297,72.748764,62.913155,445.542267,50.150002,42.934578,...,50.707050,149.682190,24.108273,35.950001,10.095658,52.131248,35.410084,50.891998,185.761429,41.161484
2016-01-05,14.05,6.65,325.375366,330.330933,79.987976,73.528900,75.487160,441.901855,50.849998,44.104424,...,52.461643,159.612122,24.191528,36.250000,10.313408,53.811543,34.979778,52.116421,183.730057,43.825382
2016-01-06,14.55,6.65,328.722137,339.503387,78.321564,72.293671,71.346153,461.105072,55.700001,43.885757,...,52.525059,162.834732,25.065752,35.849998,10.558872,54.651691,34.722279,52.336189,182.574341,42.966061
2016-01-07,13.75,6.25,317.402191,321.895508,75.322021,71.275146,68.607048,439.854126,51.250000,41.196201,...,52.229107,152.957245,24.089767,33.400002,10.000643,52.635330,33.217915,50.295483,175.254532,40.001396


In [9]:
prices_df.isna().sum().sum()

0

In [10]:
prices_df = prices_df.dropna(axis=1)

In [11]:
# given the prices of stocks in portfolio, we can calculate the returns
returns = np.log(prices_df).diff(1).dropna()
returns = returns*252
returns.head()

,RENUKA,ALOKINDS,SADBHAV,TATACHEM,NIITLTD,NAVA,SURYAROSNI,JINDALPOLY,HMT,PFC,...,GAIL,COSMOFIRST,DAAWAT,HINDOILEXP,PCBL,CHAMBLFERT,CESC,NLCINDIA,TNPL,SAIL
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,10.801660,-2.007986,-4.422455,-1.142684,-7.619376,-6.959302,4.005605,-2.458802,-8.157641,-7.771912,...,-6.874188,5.259444,-4.789196,-0.350238,-5.280787,-2.826898,2.471612,-1.858616,-2.176078,-1.573372
2016-01-05,-4.444560,15.632922,-6.286669,-2.301124,-0.262363,2.687986,45.916238,-2.067485,3.493110,6.774408,...,8.572369,16.186544,0.868760,2.094190,5.377510,7.994307,-3.081078,5.991141,-2.770894,15.802969
2016-01-06,8.812087,0.000000,2.578801,6.902015,-5.305447,-4.269370,-14.217624,10.719608,22.957217,-1.252503,...,0.304433,5.037263,8.945988,-2.796156,5.927466,3.904028,-1.861931,1.060419,-1.590160,-4.990264
2016-01-07,-14.251150,-15.632922,-8.830865,-13.420749,-9.840706,-3.575601,-9.865336,-11.890065,-20.982665,-15.937434,...,-1.423906,-15.769489,-10.008253,-17.838470,-13.687911,-9.473334,-11.161632,-10.022751,-10.311349,-18.017032
2016-01-08,17.691473,2.007986,-5.730664,5.077650,2.634368,7.696383,15.514986,4.316624,2.933100,2.329929,...,1.761806,6.729467,2.120111,3.744497,2.185206,7.918973,1.971486,3.592273,4.244646,4.559997


In [12]:
returns.dropna(inplace=True)

In [13]:
returns.head()

,RENUKA,ALOKINDS,SADBHAV,TATACHEM,NIITLTD,NAVA,SURYAROSNI,JINDALPOLY,HMT,PFC,...,GAIL,COSMOFIRST,DAAWAT,HINDOILEXP,PCBL,CHAMBLFERT,CESC,NLCINDIA,TNPL,SAIL
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,10.801660,-2.007986,-4.422455,-1.142684,-7.619376,-6.959302,4.005605,-2.458802,-8.157641,-7.771912,...,-6.874188,5.259444,-4.789196,-0.350238,-5.280787,-2.826898,2.471612,-1.858616,-2.176078,-1.573372
2016-01-05,-4.444560,15.632922,-6.286669,-2.301124,-0.262363,2.687986,45.916238,-2.067485,3.493110,6.774408,...,8.572369,16.186544,0.868760,2.094190,5.377510,7.994307,-3.081078,5.991141,-2.770894,15.802969
2016-01-06,8.812087,0.000000,2.578801,6.902015,-5.305447,-4.269370,-14.217624,10.719608,22.957217,-1.252503,...,0.304433,5.037263,8.945988,-2.796156,5.927466,3.904028,-1.861931,1.060419,-1.590160,-4.990264
2016-01-07,-14.251150,-15.632922,-8.830865,-13.420749,-9.840706,-3.575601,-9.865336,-11.890065,-20.982665,-15.937434,...,-1.423906,-15.769489,-10.008253,-17.838470,-13.687911,-9.473334,-11.161632,-10.022751,-10.311349,-18.017032
2016-01-08,17.691473,2.007986,-5.730664,5.077650,2.634368,7.696383,15.514986,4.316624,2.933100,2.329929,...,1.761806,6.729467,2.120111,3.744497,2.185206,7.918973,1.971486,3.592273,4.244646,4.559997


In [14]:
mean_returns = returns.mean()
cov_matrix = returns.cov()
weights = {}
for stock in cov_matrix.index:
    weights[stock] = portfolio[portfolio['stock'] == stock]['weights'].values[0]
W_tan = pd.Series(weights, index=cov_matrix.index)
portfolio_return = np.sum(mean_returns * W_tan)


print('Portfolio return: {}'.format(portfolio_return))
print((np.exp(portfolio_return)-1)*100)

volatility = np.sqrt((W_tan.T @ cov_matrix @ W_tan)/252)
print('Portfolio volatility: {}'.format(volatility))

Portfolio return: 2.7924061349038993
1532.0241298988599
Portfolio volatility: 0.8802091579282728


In [15]:
mean_returns['RENUKA']/mean_returns['ALOKINDS']

0.5119616163388588

In [16]:
cov_matrix/252

,RENUKA,ALOKINDS,SADBHAV,TATACHEM,NIITLTD,NAVA,SURYAROSNI,JINDALPOLY,HMT,PFC,...,GAIL,COSMOFIRST,DAAWAT,HINDOILEXP,PCBL,CHAMBLFERT,CESC,NLCINDIA,TNPL,SAIL
RENUKA,0.225473,0.045346,0.026254,0.023553,0.032147,0.043376,0.049414,0.037690,0.036912,0.041804,...,0.010505,0.042118,0.039379,0.041863,0.056267,0.025160,0.025828,0.025800,0.031341,0.054058
ALOKINDS,0.045346,0.360395,0.027736,0.023516,0.036168,0.027522,0.045860,0.036805,0.034370,0.033232,...,0.013620,0.036220,0.042234,0.043407,0.034869,0.038991,0.014405,0.022055,0.021221,0.045068
SADBHAV,0.026254,0.027736,0.145056,0.023879,0.028643,0.018224,0.038193,0.035825,0.024940,0.037557,...,0.022339,0.038885,0.040110,0.033680,0.041909,0.018192,0.028861,0.018259,0.025651,0.041869
TATACHEM,0.023553,0.023516,0.023879,0.066029,0.025844,0.029363,0.034451,0.031092,0.025720,0.031345,...,0.017246,0.034904,0.024424,0.027883,0.036531,0.023095,0.023813,0.014418,0.021001,0.040411
NIITLTD,0.032147,0.036168,0.028643,0.025844,0.172081,0.039121,0.054946,0.037361,0.032718,0.036582,...,0.012892,0.045515,0.044809,0.055415,0.057219,0.030817,0.031400,0.021551,0.033834,0.045084
NAVA,0.043376,0.027522,0.018224,0.029363,0.039121,0.159887,0.050350,0.051076,0.033595,0.039366,...,0.021478,0.041794,0.043330,0.039952,0.058172,0.031265,0.031591,0.027741,0.035493,0.053383
SURYAROSNI,0.049414,0.045860,0.038193,0.034451,0.054946,0.050350,0.202792,0.055797,0.054420,0.057370,...,0.031762,0.063106,0.067962,0.071366,0.095299,0.043947,0.040218,0.026926,0.050501,0.075657
JINDALPOLY,0.037690,0.036805,0.035825,0.031092,0.037361,0.051076,0.055797,0.174623,0.046613,0.046451,...,0.032672,0.063815,0.051580,0.051503,0.055460,0.034962,0.036357,0.026534,0.046418,0.061255
HMT,0.036912,0.034370,0.024940,0.025720,0.032718,0.033595,0.054420,0.046613,0.174727,0.035182,...,0.027210,0.038368,0.037734,0.041862,0.049415,0.028214,0.016692,0.018290,0.024210,0.044118
PFC,0.041804,0.033232,0.037557,0.031345,0.036582,0.039366,0.057370,0.046451,0.035182,0.140781,...,0.028319,0.046397,0.043849,0.042192,0.058798,0.027907,0.038979,0.026838,0.037801,0.070199


In [17]:
np.sqrt((W_tan.T @ cov_matrix @ W_tan)/252)

0.8802091579282728

In [18]:
portfolio_return

2.7924061349038993

In [19]:
print(portfolio['weights'].shape)
print(mean_returns.shape)
result = np.dot( mean_returns, portfolio['weights'])

(40,)
(40,)


In [20]:
(np.exp(result) - 1)*100

1532.0241298988599

In [21]:
weights_series = pd.Series(portfolio['weights'].values, index=portfolio['stock'].values)
mean_returns_series = pd.Series(mean_returns, index=portfolio['stock'].values)
result = weights_series.dot(mean_returns_series)


In [22]:
weights_series.sum()

0.9999999999999988